# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [25]:
import os, collections
import pandas as pd

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [26]:
oldContents = os.listdir("../_talks/")
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
print("Imported generator `talks.tsv` file:")
talks

Imported generator `talks.tsv` file:


,title,type,assesses,educationalLevel,url_slug,venue,date,location,talk_url,description,tags
0,Doctoral Defense,Talk,Stroke,intermediate,Dissertation,Global (Zoom),2020-10-09,"Kansas City KS, USA",https://docs.google.com/presentation/d/1kmUq7k...,Successfully defended my doctoral thesis with ...,"Thesis,Defense,Stroke,Rat,Motor,Recovery,Spike..."
1,Mechanisms of Recovery from Motor Stroke,Talk,Stroke,advanced,CMU,Carnegie Mellon University (Zoom),2020-11-10,"Pittsburgh PA, USA",https://docs.google.com/presentation/d/1BAc7-n...,Low-frequency (0.1-4 Hz) neural activity durin...,"CMU,Sensorimotor,BIBO,Rat,Reaching,Spike,Dynamics"
2,State Machine for Low-Latency High-Fidelity Sp...,Talk,Spike Detection,advanced,FSM,University of Kansas Medical Center,2019-01-21,"Kansas City KS, USA",NaN,I made it to the final round of the KUMC three...,"3MT,KUMC,Spike,Detection,FPGA,Intan,FSM,Closed..."
3,Neuroplasticity and Neurorehabilitation,Talk,Rehabilitation,intermediate,PlasticityLecture,Rockhurst,2017-06-08,"Kansas City MO, USA",NaN,I gave an invited guest lecture describing pri...,"Lecture,Rockhurst,BCI,Rehabilitation,Therapy"
4,Capitol Graduate Research Summit,Poster,BCI,advanced,CGRS,State Capitol Building,2017-06-21,"Topeka KS, USA",NaN,I presented a poster on a Hidden Markov Model ...,"CGRS,Topeka,Poster,Anesthetized,Rat,Markov,Model"


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [27]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [28]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    tags = item.tags.split(",")
    
    md = "---\ntitle: \""   + item.title + '"\n'

    md += "collection: talks" + "\n"

    md += "category: talks" + "\n"

    md += 'layout: talk\n'
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "/\n"

    md += "redirect_from:\n"

    md += "  - \"/talks/" + html_filename + ".html\"\n"
    
    md += "  - \"/talks/" + html_filename + "\"\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'

    md += "author_profile: true\n"

    md += "assesses: " + item.assesses + "\n"

    md += "educationalLevel: " + item.educationalLevel + "\n"

    ## Add tags in lowercase (they are delimited using "," in the tsv file)
    md += "tags:\n"
    for tag in tags:
        md += "  - " + html_escape(tag.lower()) + "\n"

    md += "keywords:\n"
    for tag in tags:
        md += "  - " + html_escape(tag.lower()) + "\n"

    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [29]:
newContents = os.listdir("../_talks/")
if collections.Counter(oldContents) == collections.Counter(newContents):
    print("No new `talks` files added.")
    print("Existing files:")
    for f in oldContents:
        print("\t" + f)
else:
    print("Generated new `talks` files:")
    for f in newContents:
        print("\t" + f)

No new `talks` files added.
Existing files:
	2017-06-08-PlasticityLecture.md
	2017-06-21-CGRS.md
	2019-01-21-FSM.md
	2020-10-09-Dissertation.md
	2020-11-10-CMU.md
